In [12]:
#load the dataset
import pandas as pd

df = pd.read_csv(r'C:\Users\thobi\Downloads\archive (1)\WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [14]:
#Check for any incosistencies
df.info()
df.isnull().sum()
df.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


In [16]:
#Messy data, time to clean it
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')


In [18]:
df['TotalCharges'].isnull().sum()


11

There's still 11 missing values, this means that the new customers haven't been charged yet. So I'll set the total charges to zero.

In [21]:
df['TotalCharges'] = df['TotalCharges'].fillna(0)


In [23]:
#Converting churn to binary
df['churn_flag'] = df['Churn'].map({'Yes': 1, 'No': 0})
#Churn rate
df['churn_flag'].mean()


0.2653698707936959

We just found that about 26% of customers have churned! This is significant and requires deeper analysis.

In [30]:
#Churn by contract type
df.groupby('Contract')['churn_flag'].mean().sort_values(ascending=False)


Contract
Month-to-month    0.427097
One year          0.112695
Two year          0.028319
Name: churn_flag, dtype: float64

Month-to-month customers churn more compared to customers with one or two year contracts.


In [41]:
#Churn vs Tenure
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
df.groupby(pd.cut(df['tenure'], bins=[0, 12, 24, 48, 72]))['churn_flag'].mean()


tenure
(0, 12]     0.476782
(12, 24]    0.287109
(24, 48]    0.203890
(48, 72]    0.095132
Name: churn_flag, dtype: float64

New customers are more likely to churn early compared to old customers.

In [44]:
#Charges vs Churn
df.groupby('churn_flag')[['MonthlyCharges', 'TotalCharges']].mean()


,MonthlyCharges,TotalCharges
churn_flag,,
0,61.265124,2549.911442
1,74.441332,1531.796094


Churned customers often have higher monthly charges but lower lifetime value.

# Insight 1
Customers with higher monthly charges show a higher churn rate, suggesting price sensitivity or perceived lack of value.
# Insight 2
Early-stage customers are more likely to churn, likely due to weaker product attachment and incomplete onboarding.
# Hypothesis
Customers who are new AND paying high monthly charges are at the highest risk of churn.


In [51]:
#Testing the hypothesis
df['tenure_group'] = pd.cut(
    df['tenure'],
    bins=[0, 12, 24, 48, 72],
    labels=['0-1yr', '1-2yr', '2-4yr', '4-6yr']
)

pd.pivot_table(
    df,
    values='churn_flag',
    index='tenure_group',
    columns=pd.cut(df['MonthlyCharges'], bins=3),
    aggfunc='mean'
)


MonthlyCharges,"(18.15, 51.75]","(51.75, 85.25]","(85.25, 118.75]"
tenure_group,,,
0-1yr,0.318957,0.560890,0.740741
1-2yr,0.109626,0.294271,0.526316
2-4yr,0.056863,0.204340,0.344633
4-6yr,0.026596,0.057543,0.156008


Customers with the highest churn have low tenure AND higher monthly charges.
## Conclusion
Churn is driven by a combination of pricing and customer maturity. New customers with higher monthly charges exhibit the highest churn rates, suggesting that perceived value during early onboarding is a critical retention lever.
## Suggestions
Our analysis showed churn is highest among new customers with high monthly charges. Rather than broad discounts, I’d recommend targeted early-tenure retention strategies — such as temporary price relief or incentives to move to annual contracts, combined with stronger onboarding to reinforce value during the first 90 days.

# Churn Risk Scoring System

In [60]:
#Monthly charges Buckets
high_charge = df['MonthlyCharges'].quantile(0.7)
mid_charge = df['MonthlyCharges'].quantile(0.3)

def charge_score(x):
    if x >= high_charge:
        return 40
    elif x >= mid_charge:
        return 20
    else:
        return 0

df['charge_score'] = df['MonthlyCharges'].apply(charge_score)


In [62]:
#Tenure Score
def tenure_score(x):
    if x < 12:
        return 35
    elif x < 24:
        return 20
    else:
        return 0

df['tenure_score'] = df['tenure'].apply(tenure_score)


In [64]:
#Contract score
contract_map = {
    'Month-to-month': 25,
    'One year': 10,
    'Two year': 0
}

df['contract_score'] = df['Contract'].map(contract_map)


In [66]:
#Final churn risk score
df['churn_risk_score'] = (
    df['charge_score'] +
    df['tenure_score'] +
    df['contract_score']
)


In [68]:
#Segment customers by risk
def risk_segment(score):
    if score >= 70:
        return 'High Risk'
    elif score >= 40:
        return 'Medium Risk'
    else:
        return 'Low Risk'

df['risk_segment'] = df['churn_risk_score'].apply(risk_segment)


In [70]:
#Validate the model
df.groupby('risk_segment')['churn_flag'].mean()


risk_segment
High Risk      0.596154
Low Risk       0.041213
Medium Risk    0.257886
Name: churn_flag, dtype: float64

High risk customers have the highest churn rate. The churn risk model successfully stratifies customers. Nearly 60% of high-risk customers churn, compared to just 4% in the low-risk segment. This allows retention teams to focus efforts where impact is highest.